In [4]:
### 医疗领域数据集生成
import json
import re
with open("/home/work/liuytest/大论文/医疗数据集/tongyi_query_pair_query.json",'r') as file:
    data = json.load(file)



schema_ldbc_info = ""
with open("/home/work/liuytest/大论文/医疗数据集/schema_medical_info.json", "r",encoding="utf-8") as file:
    schema_ldbc_info= "已知数据库的schema信息如下：\n"+ json.dumps(json.load(file), ensure_ascii=False) 
        
with open("/home/work/liuytest/大论文/医疗数据集/new_query_pair_train_tongyi.json",'r') as file:
    data = json.load(file)

new_data = []     
for row in data:
    new_row ={}
    instruction =  row['instruction'].replace(",的节点","的节点")
     # 使用正则表达式删除带有"得分为"的内容
    instruction = re.sub(r"得分为-0.[\d]+,", "", instruction)
    instruction = re.sub(r", \"描述\": \".+\"", "", instruction)
    instruction = re.sub(r", \"预防方法\": \".+\"", "", instruction)
    instruction = re.sub(r", \"病因\": \".+\"", "", instruction)
    # instruction = instruction.replce('\n请将以下自然语言翻译为对该数据库的Cypher查询:\n','')
    new_row["instruction"] = '你是一位自然语言转Cypher的专家,请将以下自然语言转换为Cypher查询语句,\n例如自然语言描述为查询全部节点,你返回 match (n) return n\n'+instruction
    new_row["output"] = row["output"]
    new_data.append(new_row)
        
with open('medical_with_vector_prompt.json', 'w') as file:
    json.dump(new_data,file,indent=4 ,ensure_ascii=False)

    
with open("/home/work/liuytest/大论文/医疗数据集/new_query_pair_train_tongyi.json",'r') as file:
    data = json.load(file) 
new_data = []         
for row in data:
    new_row ={}
    instruction =  row['instruction'].replace(",的节点","的节点")
     # 使用正则表达式删除带有"得分为"的内容
    instruction = re.sub(r"得分为-0.[\d]+,", "", instruction)
    instruction = re.sub(r", \"描述\": \".+\"", "", instruction)
    instruction = re.sub(r", \"预防方法\": \".+\"", "", instruction)
    instruction = re.sub(r", \"病因\": \".+\"", "", instruction)
    
    # instruction = instruction.replce('\n请将以下自然语言翻译为对该数据库的Cypher查询:\n','')
    new_row["instruction"] = '你是一位自然语言转Cypher的专家,请将以下自然语言转换为Cypher查询语句,\n例如自然语言描述为查询全部节点,你返回 match (n) return n\n'+schema_ldbc_info+"\n"+instruction
    new_row["output"] = row["output"]
    new_data.append(new_row)
        
with open('medical_with_schema_vector_prompt.json', 'w') as file:
    json.dump(new_data,file,indent=4 ,ensure_ascii=False)
len(new_data)     

908

In [ ]:
## 每种不同的prompt都以 7:3 的比例划分  在不同的模型上训练
### 测试数据集需要在 没有训练和训练过的模型上都测试效果
### 我们在 qwen8B llama7B chatglm6B  gemma9B 模型上用四种 prompt来训练
### 我们在  qwen8B llama7B chatglm6B  gemma9B  训练前和训练后都要做推理评测实验  这是我们的核心试验
### 7比3 的数据集在不同规模的大模型，这里我们只做我们的数据集 Qwen0.5 到千问 32B上都要训练
import json
import random

list_index = [i for i in range(len(data))]
train_data_0_7_index = random.sample(list_index,int(len(list_index)*0.7))

test_data_0_7_index = [ i for i in list_index if i not in set(train_data_0_7_index)]
        
with open("ldbc_with_schema_vector_prompt.json") as file:
    data = json.load(file) 
train_data_7_3 = [data[i] for i in train_data_0_7_index]
test_data_7_3 = [data[i] for i in test_data_0_7_index]           
with open('ldbc_with_schema_vector_prompt_train_7_3.json', 'w') as file:
    json.dump(train_data_7_3,file,indent=4 ,ensure_ascii=False)  
with open('ldbc_with_schema_vector_prompt_test_7_3.json', 'w') as file:
    json.dump(test_data_7_3,file,indent=4 ,ensure_ascii=False) 